<a href="https://colab.research.google.com/github/Jaehwi-So/DeepLearning_Study/blob/main/DL01_%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import numpy as np

class SimpleMLP:
    # 활성화 함수
    def relu(self, x):
        return np.maximum(0, x)

    def linear(self, x):
        return x


    # 입력값의 차원, Hidden layer의 개수, 출력값의 차원을 기반으로 Weight와 Bias를 생성하고 초기화
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_W = np.random.randn(input_size, hidden_size) * 0.01
        self.hidden_b = np.zeros((1, hidden_size))
        self.output_W = np.random.randn(hidden_size, output_size) * 0.01
        self.output_b = np.zeros((1, output_size))


    # Forward Propagation
    def forward(self, X):
        self.hidden_Z = np.dot(X, self.hidden_W) + self.hidden_b # 은닉층의 입력 Z : 가중합(입력과 가중치의 내적에 편향을 더한다)
        self.hidden_A = self.relu(self.hidden_Z) # 은닉층의 출력 A : ReLu 활성 함수 적용
        self.output_Z = np.dot(self.hidden_A, self.output_W) + self.output_b # 출력층의 입력 Z : 가중합(은닉층의 출력과 가중치의 내적에 편향을 더한다)
        self.output_A = self.linear(self.output_Z) # 출력층의 출력 A : Linear 활성 함수 적용
        return self.output_A

    # Backward Propagation
    def backward(self, X, y, output, learning_rate):
        m = y.shape[0]

        # 출력층의 Gradient 계산 : Loss에 대한 출력의 Gradient 계산
        d_oZ = output - y.reshape(-1, 1)

        # 출력층 가중치 Gradient 계산 : 출력층의 Gradient와 (이전 Layer에서 들어온)은닉층의 출력(A)을 사용하여 출력층의 가중치를 갱신하기 위한 Gradient를 계산한다
        d_oW = (1/m) * np.dot(self.hidden_A.T, d_oZ)
        d_ob = (1/m) * np.sum(d_oZ, axis=0, keepdims=True)

        # 은닉층의 Gradient 계산
        d_hA = np.dot(d_oZ, self.output_W.T) # 출력층의 그래디언트를 은닉층의 출력에 대한 그래디언트로 변환
        d_hZ = d_hA * (self.hidden_Z > 0).astype(float)  # ReLU의 미분

        # 은닉층의 가중치 Gradient 계산 : 은닉층의 Gradient와 (이전 Layer에서 들어온)입력층의 출력을 사용하여 은닉층의 가중치를 갱신하기 위한 Gradient를 계산한다.
        d_hW = (1/m) * np.dot(X.T, d_hZ)
        d_hb = (1/m) * np.sum(d_hZ, axis=0, keepdims=True)

        # 역전파를 통해 구한 Gradient를 통해 가중치와 편향을 갱신한다.
        self.hidden_W -= learning_rate * d_hW
        self.hidden_b -= learning_rate * d_hb
        self.output_W -= learning_rate * d_oW
        self.output_b -= learning_rate * d_ob


    # MLP 학습
    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward(X) # 1. Forward Propagation
            loss = np.mean((output - y.reshape(-1, 1)) ** 2) # 2. Loss 계산
            self.backward(X, y, output, learning_rate) # 3. Backward Propagation
            if epoch % 10 == 0:
              print(f"Epoch {epoch}, Loss: {loss}") # 4. epochs만큼 반복

    def predict(self, X):
        return self.forward(X)


In [53]:
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 데이터 로드 및 전처리
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# 데이터 스케일링 (특성 표준화)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터셋을 훈련 세트와 테스트 세트로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 초기화 및 학습
input_size = X_train.shape[1]
hidden_size = 10
output_size = 1

mlp = SimpleMLP(input_size, hidden_size, output_size)
mlp.train(X_train, y_train, epochs=300, learning_rate=0.001)

# 예측 및 성능 평가
predictions = mlp.predict(X_test)
mse = np.mean((predictions.flatten() - y_test) ** 2)
print(f"Test MSE: {mse}")

Epoch 0, Loss: 29711.2862650617
Epoch 10, Loss: 29237.525418560024
Epoch 20, Loss: 28562.886510520086
Epoch 30, Loss: 21413.230581915846
Epoch 40, Loss: 8793.256977391542
Epoch 50, Loss: 5850.951975848452
Epoch 60, Loss: 3225.2907439447113
Epoch 70, Loss: 2894.409444978212
Epoch 80, Loss: 2877.0781729440178
Epoch 90, Loss: 2871.206963478034
Epoch 100, Loss: 2867.330704542642
Epoch 110, Loss: 2863.7487121485074
Epoch 120, Loss: 2860.312657080264
Epoch 130, Loss: 2857.4028471887987
Epoch 140, Loss: 2854.793149776095
Epoch 150, Loss: 2852.3170884268065
Epoch 160, Loss: 2849.8978937607126
Epoch 170, Loss: 2847.5430914665026
Epoch 180, Loss: 2845.125914529291
Epoch 190, Loss: 2842.714855728913
Epoch 200, Loss: 2840.4368772490825
Epoch 210, Loss: 2838.2586783932643
Epoch 220, Loss: 2836.185954170785
Epoch 230, Loss: 2834.171671055218
Epoch 240, Loss: 2832.2775768387737
Epoch 250, Loss: 2830.372609236236
Epoch 260, Loss: 2828.446024777999
Epoch 270, Loss: 2826.4406772245547
Epoch 280, Loss: 2